# 1. A simple MATLAB function

In [ ]:
import numpy as np

# identity matrix
A = np.eye(5, dtype=int)

print(A)

# 2. Linear regression with one variable
## 2.1 Plotting the data

In [ ]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# load and analyze data
data_file_1 = Path(os.path.abspath('')).absolute() / "data" / "ex1data1.txt"
data_set_1 = pd.read_csv(data_file_1, header=None, names=["Population", "Profit"])
# print(data)
# print(data.head())
# print(data.describe())

# make plot of dataframe. by default, matplotlib backend is used.
data_set_1.plot(kind='scatter', x='Population', y='Profit')
plt.show()

# extract data
x = np.matrix(data_set_1.iloc[:, :1])
y = np.matrix(data_set_1.iloc[:, 1:])
print(x)
print(y)

## 2.2 Gradient Descent
### 2.2.1 Update Equations
### 2.2.2 Implementation

In [ ]:
# number of training examples
m = len(x)
# insert np.ones(m) to column 0
X = np.insert(x, 0, np.ones(m), axis=1)
print(X)
# init theta with 0s
theta = np.matrix(np.zeros((2,1)))
print(theta)
iterations = 1500
alpha = 0.01

X.shape, y.shape, theta.shape


### 2.2.3 Computing the cost function

In [ ]:
# compute cost
def compute_cost(X, y, theta):
    m = len(y)
    sum = 0
    for i in range(0, m):
        diff = np.matrix(X[i]) * theta - y[i]
        sum += diff.item() ** 2
    return 1 / (2 * m) * sum

print(compute_cost(X, y, theta))
# init theta with [[-1],[2]]
print(compute_cost(X, y, np.matrix([[-1],[2]])))

# simpler vectorized method
def vectorized_compute_cost(X, y, theta):
    m = len(y)
    h = X * theta
    diff = h - y
    res = 1 / (2 * m) * np.sum(np.square(diff)) 
    return res

print(vectorized_compute_cost(X, y, theta))
# init theta with [[-1],[2]]
print(vectorized_compute_cost(X, y, np.matrix([[-1],[2]])))